## This section is a mess. I tried something from an online site because I felt confident about it. 

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, UpSampling2D, concatenate, Input

input_height=416
input_width=608
n_classes=51

# Model definition
img_input = Input(shape=(input_height,input_width , 3 ))

# Define the model architecture

conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(img_input)
conv1 = Dropout(0.2)(conv1)
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
pool1 = MaxPooling2D((2, 2))(conv1)

conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
conv2 = Dropout(0.2)(conv2)
conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
pool2 = MaxPooling2D((2, 2))(conv2)

# Decoding 
conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
conv3 = Dropout(0.2)(conv3)
conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)

up1 = concatenate([UpSampling2D((2, 2))(conv3), conv2], axis=-1)
conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(up1)
conv4 = Dropout(0.2)(conv4)
conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv4)

up2 = concatenate([UpSampling2D((2, 2))(conv4), conv1], axis=-1)
conv5 = Conv2D(32, (3, 3), activation='relu', padding='same')(up2)
conv5 = Dropout(0.2)(conv5)
conv5 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv5)

out = Conv2D( n_classes, (1, 1) , padding='same')(conv5)


model = 


2023-03-11 17:13:56.986693: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-11 17:14:04.330153: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
model.train( 
    train_images =  "dataset1/images_prepped_train/",
    train_annotations = "dataset1/annotations_prepped_train/",
    checkpoints_path = "checkpoints/vgg_unet_1" , epochs=5,
    #initialize_weights=True
)

Verifying training dataset


100%|██████████| 367/367 [00:03<00:00, 121.03it/s]


Dataset verified! 
Epoch 1/5
  8/512 [..............................] - ETA: 1:08:08 - loss: 4.2204 - accuracy: 0.0089

KeyboardInterrupt: 

In [ ]:
out = model.predict_segmentation(
    inp="dataset_path/images_prepped_test/0016E5_07965.png",
    out_fname="output.png"
)

## Now, going to try again. This time reading what I'm doing.

### Classificatoin using VGG16 demo

In [2]:
import tensorflow as tf
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.applications.resnet import preprocess_input, decode_predictions
from tensorflow.keras.applications import VGG16
model = VGG16()

# This is the architecture of the model
print(model.summary())
# For some reason it looks smaller. Okay, it's just the output being truncated. 

# Alternatively, I can draw the architecture if I've set up pydot.
# If you have, uncomment the bottom 2 lines to see magic.
# Will not break the program otherwise.
# from keras.utils.vis_utils import plot_model
# plot_model(model) #, to_file='vgg.png')


# Load image and convert to array.
image = img_to_array(load_img('Data/train2014/COCO_train2014_000000000025.jpg', target_size=(224, 224)))
# Convert to compatible shape for the model
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
# subtarct the mean rgb value, which has been computed from the training images.
# This is how the imagenet training data was prepared that was used to train the VGG16.
image = preprocess_input(image)

# Perform a prediction on the image
output = model.predict(image)

label = decode_predictions(output) # This will give a list of list of tuples (class ID??, class name, confidence between 0-1)
# The first label is the one with the highest probability.
label = label[0][0]
# Printing it
print('\n%s (%.2f%%)' % (label[1], label[2]*100))

2023-03-11 18:48:54.377937: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-11 18:48:59.543581: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

That worked fine. The giraffe was classified as an ostrich, but it's cool. For now.

I've understood a little of the workflow of taking an image and processing it for prediction.
And what the prediction looks like.

### Now let's try to create a unet model using vgg16 as a foundation for the encoding layers.

#### Unet model

In [3]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate

def unet_vgg16(input_shape, num_classes):
    # Define the input layer
    inputs = Input(input_shape)

    # Load the VGG16 model pre-trained on ImageNet
    vgg16 = VGG16(include_top=False, weights='imagenet', input_tensor=inputs)

    # Freeze the weights of the VGG16 model
    # This would help in just training the decoder layers.
    for layer in vgg16.layers:
        layer.trainable = False

    # Add the first upsampling block
    up1 = UpSampling2D(size=(2, 2))(vgg16.output)
    conv6 = Conv2D(512, (3, 3), activation='relu', padding='same')(up1)
    conv6 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv6)
    conv6 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv6)
    concat6 = Concatenate()([vgg16.get_layer('block4_conv3').output, conv6])
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(concat6)

    # Add the second upsampling block
    up2 = UpSampling2D(size=(2, 2))(conv6)
    conv7 = Conv2D(256, (3, 3), activation='relu', padding='same')(up2)
    conv7 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv7)
    conv7 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv7)
    concat7 = Concatenate()([vgg16.get_layer('block3_conv3').output, conv7])
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(concat7)

    # Add the third upsampling block
    up3 = UpSampling2D(size=(2, 2))(conv7)
    conv8 = Conv2D(128, (3, 3), activation='relu', padding='same')(up3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)
    concat8 = Concatenate()([vgg16.get_layer('block2_conv2').output, conv8])
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(concat8)

    # Add the fourth upsampling block
    up4 = UpSampling2D(size=(2, 2))(conv8)
    conv9 = Conv2D(64, (3, 3), activation='relu', padding='same')(up4)
    conv9 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv9)
    concat9 = Concatenate()([vgg16.get_layer('block1_conv2').output, conv9])
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(concat9)

    # Add the output layer
    outputs = Conv2D(num_classes, (1, 1), activation='softmax')(conv9)

    # Create and compile the model
    model = Model(inputs=vgg16.input, outputs=outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

#### Image preprocess pipeline

In [4]:
# Image processing pipeline for network input
import os

def preprocess_image(image_dir, mask_dir, input_shape):

    # Load the image.
    img = Image.open(image_dir)
    mask = Image.open(mask_path)

    # Resize the images to input_shape.
    img = img.resize(input_shape)
    mask = mask.resize(input_shape)

    # Convert the images to numpy arrays.
    img_array = np.asarray(img)
    mask_array = np.asarray(mask)

    # Normalize the pixel values of the input images and convert to array.
    img_array = np.array(img_array / 255.0)

    # Convert the mask images to binary masks and convert to array.
    mask_array = np.array((mask_array > 0).astype(np.uint8))

    return img_to_array, mask_array